In [1]:
import pandas as pd

In [2]:
df_val=pd.read_csv("dataset/telemetry_for_operations_validation.csv")
df_lables=pd.read_csv("dataset/operations_labels_training.csv")

In [3]:
# Check if the format is 'YYYY-MM-DD HH:MM:SS'
def check_format(date_str):
    # Try to convert to datetime with a specific format
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return 'datetime'
    except ValueError:
        try:
            pd.to_datetime(date_str, format='%Y-%m-%d')
            return 'date'
        except ValueError:
            return 'invalid'

# Apply the function to the 'create_dt' column
df_val['format'] = df_val['create_dt'].apply(check_format)

In [5]:
# Display the rows with 'date' or 'invalid' format
df_val=df_val.drop(df_val[df_val['format'] != 'datetime'].index)


In [6]:
# def assign_labels(group):
#     object_labels = df_lables[df_lables['mdm_object_name'] == group.name]
    
#     for _, row in object_labels.iterrows():
#         mask = (group['create_dt'] > row['start_time']) & (group['create_dt'] < row['end_time'])
#         group.loc[mask, 'operation_kind_id'] = row['operation_kind_id']
#         group.loc[mask, 'name'] = row['name']
    
#     return group

# # Apply the function to each group of mdm_object_name
# train = df_val.groupby('mdm_object_name').apply(assign_labels)

# # Verify the results
# print(train['operation_kind_id'].value_counts(dropna=False))

# Save the updated DataFrame
df_val.to_csv('train_with_labels_final.csv', index=False)

In [7]:
df=pd.read_csv("train_with_labels_final.csv")


In [8]:
df.dropna(inplace=True)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# encoded_data = encoder.fit_transform(df[['name']])
# encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['name']))

# # concatenate the encoded data with the rest of the DataFrame
# df = pd.concat([df.drop('name', axis=1), encoded_df], axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



In [9]:
df['create_dt'] = pd.to_datetime(df['create_dt'], format='%Y-%m-%d %H:%M:%S')

In [10]:
df['create_dt_hour'] = df['create_dt'].dt.hour
df['create_dt_minute'] = df['create_dt'].dt.minute
df['create_dt_second'] = df['create_dt'].dt.second

In [11]:
df['create_dt_hms'] = df['create_dt'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

In [12]:
df['create_dt_hms'] = df['create_dt_hms'].astype('float64')

In [13]:
df['create_dt_hms'] = pd.to_numeric(df['create_dt_hms'])

In [14]:
df.drop(['create_dt'], axis=1, inplace=True)
df.drop(['create_dt_hour'], axis=1, inplace=True)
df.drop(['create_dt_minute'], axis=1, inplace=True)
df.drop(['create_dt_second'], axis=1, inplace=True)
df.drop(['format'], axis=1, inplace=True)

In [15]:
print(df.dtypes)

mdm_object_name        int64
mdm_model_id           int64
lon                  float64
lat                  float64
alt                    int64
speed_gps            float64
direction            float64
accel_forward_nn     float64
accel_braking_nn     float64
accel_angular_nn     float64
accel_vertical_nn    float64
create_dt_hms        float64
dtype: object


In [16]:
df.dropna(inplace=True)

In [17]:
# split data into training and testing sets
X = df.drop('operation_kind_id', axis=1)  # features
y = df['operation_kind_id']  # target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# create a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# train the model
rf.fit(X_train, y_train)

# make predictions on the test set
y_pred = rf.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

KeyError: "['operation_kind_id'] not found in axis"